# 5) Quantal response equilibria

Gambit implements the idea of [McKPal95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) and [McKPal98](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) to compute Nash equilibria via path-following a branch of the logit quantal response equilibrium (LQRE) correspondence using the function `logit_solve`.
As an example, we will consider an asymmetric matching pennies game from [Och95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) as analyzed in [McKPal95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts).

In [1]:
import pygambit as gbt

In [2]:
g = gbt.Game.from_arrays(
        [[1.1141, 0], [0, 0.2785]],
        [[0, 1.1141], [1.1141, 0]],
        title="Ochs (1995) asymmetric matching pennies as transformed in McKelvey-Palfrey (1995)"
)
gbt.nash.logit_solve(g).equilibria[0]

[[0.5000000234106035, 0.49999997658939654], [0.19998563837426647, 0.8000143616257336]]

`logit_solve` returns only the limiting (approximate) Nash equilibrium found.
Profiles along the QRE correspondence are frequently of interest in their own right.
Gambit offers several functions for more detailed examination of branches of the QRE correspondence.

The function `logit_solve_branch` uses the same procedure as `logit_solve`, but returns a list of LQRE profiles computed along the branch instead of just the limiting approximate Nash equilibrium.

In [3]:
qres = gbt.qre.logit_solve_branch(g)
len(qres)

193

In [4]:
qres[0].profile

[[0.5, 0.5], [0.5, 0.5]]

In [5]:
qres[5].profile

[[0.5182276540742868, 0.4817723459257562], [0.49821668880066783, 0.5017833111993909]]

`logit_solve_branch` uses an adaptive step size heuristic to find points on the branch.
The parameters `first_step` and `max_accel` are used to adjust the initial step size and the maximum rate at which the step size changes adaptively.
The step size used is computed as the distance traveled along the path, and, importantly, not the distance as measured by changes in the precision parameter lambda.
As a result the lambda values for which profiles are computed cannot be controlled in advance.

In some situations, the LQRE profiles at specified values of lambda are of interest.
For this, Gambit provides `logit_solve_lambda`.
This function provides accurate values of strategy profiles at one or more specified values of lambda.

In [6]:
qres = gbt.qre.logit_solve_lambda(g, lam=[1, 2, 3])
qres[0].profile

[[0.5867840364385154, 0.4132159635614846], [0.4518070316997103, 0.5481929683002897]]

In [7]:
qres[1].profile

[[0.6175219458400859, 0.3824780541599141], [0.3719816648492249, 0.6280183351507751]]

In [8]:
qres[2].profile

[[0.6168968501329284, 0.3831031498670716], [0.31401636202001226, 0.6859836379799877]]

LQRE are frequently taken to data by using maximum likelihood estimation to find the LQRE profile that best fits an observed profile of play.
This is provided by the function `logit_estimate`.
We replicate the analysis of a block of the data from [Och95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) for which [McKPal95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) estimated an LQRE.

In [9]:
data = g.mixed_strategy_profile([[128*0.527, 128*(1-0.527)], [128*0.366, 128*(1-0.366)]])
fit = gbt.qre.logit_estimate(data)
type(fit)

pygambit.qre.LogitQREMixedStrategyFitResult

The returned `LogitQREMixedStrategyFitResult` object contains the results of the estimation.
The results replicate those reported in [McKPal95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts), including the estimated value of lambda, the QRE profile probabilities, and the log-likelihood.

Because `data` contains the empirical counts of play, and not just frequencies, the resulting log-likelihood is correct for use in likelihoood-ratio tests.
<sup>[[1](#f1)]</sup>

In [10]:
print(fit.lam)
print(fit.profile)
print(fit.log_like)

1.8456097536855862
[[0.615651314427859, 0.3843486855721409], [0.38329094004562914, 0.6167090599543709]]
-174.76453191087447


All of the functions above also support working with the agent LQRE of [McKPal98](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts).
Agent QRE are computed as the default behavior whenever the game has a extensive (tree) representation.

For `logit_solve`, `logit_solve_branch`, and `logit_solve_lambda`, this can be overriden by passing `use_strategic=True`;
this will compute LQRE using the reduced strategy set of the game instead.

Likewise, `logit_estimate` will perform estimation using agent LQRE if the data passed are a `MixedBehaviorProfile`, and will return a `LogitQREMixedBehaviorFitResult` object.

**Footnotes:**

<a id="f1"></a> The log-likelihoods quoted in [McKPal95](https://gambitproject.readthedocs.io/en/latest/biblio.html#general-game-theory-articles-and-texts) are exactly a factor of 10 larger than those obtained by replicating the calculation.